In [ ]:
%gsutil cp gs://aurora-bay-data/aurora_bay_data.sql .

In [ ]:
# sql_engine: bigquery
# output_variable: df
# start _sql
_sql = """
CREATE OR REPLACE MODEL
  `aurora_bay_data.embedding_model` REMOTE
WITH CONNECTION `projects/1061068428496/locations/us/connections/aurora-connector` OPTIONS (ENDPOINT = 'text-embedding-005');
""" # end _sql
from google.colab.sql import bigquery as _bqsqlcell
df = _bqsqlcell.run(_sql)
df

In [ ]:
# sql_engine: bigquery
# output_variable: df
# start _sql
_sql = """
CREATE OR REPLACE TABLE
  `aurora_bay_data.aurora_data_with_embeddings` AS
SELECT
  *
FROM
  ML.GENERATE_EMBEDDING( MODEL `aurora_bay_data.embedding_model`,
    (
    SELECT
      string_field_0,
      string_field_1,
      CONCAT(string_field_0, string_field_1) AS content
    FROM
      `aurora_bay_data.aurora_data_raw`) );
""" # end _sql
from google.colab.sql import bigquery as _bqsqlcell
df = _bqsqlcell.run(_sql)
df

In [ ]:
# sql_engine: bigquery
# output_variable: df
# start _sql
_sql = """
SELECT
  query.query,
  base.string_field_0,
  base.string_field_1
FROM
  VECTOR_SEARCH( TABLE `aurora_bay_data.aurora_data_with_embeddings`,
    'ml_generate_embedding_result',
    (
    SELECT
      text_embedding,
      content AS query
    FROM
      ML.GENERATE_TEXT_EMBEDDING( MODEL `aurora_bay_data.embedding_model`,
        (
        SELECT
          'Weather' AS content)) ),
    top_k => 5,
    OPTIONS => '{"fraction_lists_to_search": 0.01}')
""" # end _sql
from google.colab.sql import bigquery as _bqsqlcell
df = _bqsqlcell.run(_sql)
df